## Lab04
### Correcting spelling errors in context

Format:
- Error
- Candidates
- Correstion
- Wrong -> Correct
- hits(accumulation)

In [1]:
import re
from collections import Counter, defaultdict
from pprint import pprint

'''Word Probability'''
def words(text): 
    return re.findall(r'\w+', text.lower())

count_word = Counter(words(open('C:/Users/asus/Downloads/nlp/big.txt').read()))
Nw = sum(count_word.values())
Pdist = {word: float(count)/Nw for word, count in count_word.items()}

def Pw(word): 
    return Pdist[word] if word in Pdist else 10/10**len(word)/Nw

In [2]:
'''Channel Probability'''
from collections import Counter
with open('C:/Users/asus/Downloads/nlp/count_1edit.txt' ,encoding='UTF-8-sig') as f:
    line = f.readlines()
    
"""Put every line in list"""
line_arr=[]
for i in range(len(line)):
     if len(line[i])>=2:
        line[i]= line[i][:-1].split('\t')
        line_arr.append(line[i])
        
"""Get key list"""
k_arr=[]
for i in range(len(line_arr)):
        k_arr.append(tuple(line_arr[i][0].split('|')))

"""Get value list"""
v_arr=[]
for i in range(len(line_arr)):
    # replace wierd words
    if line_arr[i][-1]=='' :
        print(i)
    else:
        v_arr.append(int(line_arr[i][-1]))

"""Zip them to dict"""
word_dict = dict(zip(k_arr, v_arr))

#(w,c)= 1,2,3次的次數
Nr = (Counter(word_dict.values()))    #新增n0進去!
#w,c 的次數 
count_wc = word_dict   #新增n0進去!
# 計算每個c出現的次數統計, 產生一個新的dict儲存
count_c = dict()   
for key, value in (count_wc.items()):
    if key[1] in count_c:
        count_c[key[1]]+=value
    else:
        count_c[key[1]]=value   
Nall= sum(Nr.values())
N0 = 26*26*26*26+2*26*26*26+26*26 - Nall   #沒有出現的組合次數
Nr_smooth = [ N0 if r==0 else (r+1)*Nr[r+1] / Nr[r] for r in range(11) ]

1586


In [3]:
def Pedit(w, c):
    if (w,c) not in word_dict: #r=0
        if c in count_c:         #有在正確的list中
            return 317/N0/count_c[c]
        else:
            return 0
    elif 0<count_wc[(w,c)] <= 10:  #需要smooth的
        r = count_wc[(w,c)]
        return Nr_smooth[r]/count_c[c]
    else:      #>10的
        return count_wc[(w,c)]/count_c[c]   #Ped(w, c) = count1(w, c)/count(c)

In [4]:
def next_states(state):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    (L, R, edit, prob, pedit) = state
    all_word=[]

    R0, R1 = R[0:1], R[1:]
    if edit == 2:  
        return [( L + R0,     R1, edit,   Pw(L+R0+R1) , pedit)]
    else:
        all_word.append(( L + R0,     R1, edit,   Pw(L+R0+R1) , pedit))            #noedit
        all_word.append(( L ,         R1 ,edit+1, Pw(L+R1),     pedit* Pedit(L[-1:]+R0,L[-1:]) ))  #delete
        all_word.append(( L+ R1[0], R0+R1[1:], edit+1,Pw(L+ R1[0]+ R0+R1[1:]),pedit* Pedit(R0+R1[0],R1[0]+R0))
                        if len(R0)>1 else (L+R1 , R0, edit+1, Pw(L+R0+R1),    pedit*Pedit(R0+R1, R1+R0))) #transpose
        for c in letters:
            all_word.append((L + c ,    R1, edit+1, Pw(L+c+R1),    pedit*Pedit(R0, c) )) #replace
            all_word.append((L + R0 + c,R1, edit+1, Pw(L+R0+c+R1), pedit*Pedit(R0, R0+c) )) #insert      
        return all_word

In [5]:
""" small edit and large probability, 最後取機率大的 """

def correction(word):
    states = [ ("", word, 0, 0, 1) ]
    for i in range(len(word)):
        states = [ states for state in states for states in next_states(state)]
        
    temp = defaultdict(list)
    for state in states:
        L, R, edits, pw, pedit = state
        temp[L+R].append(state)  #L+R相同的選state裡edit最小的
    states = [min(substates, key=lambda x: (x[2],x[3]*x[4]*(-1))) for wd, substates in temp.items()]    
## state[2]==0的取出 sort
    states0 = [state for state in states if states[2]==0]
    states0 = sorted(states0, key=lambda x: -1*x[3]*x[4])    
## state[2]!=0的另外分類    
    states_ = [state for state in states if states[2]!=0]
    states_ = sorted(states_, key=lambda x: -1*x[3]*x[4])
    
    states = (states0 + states_)[:500]
    return sorted(states, key=lambda x: ((-1)*x[3]*x[4],(-1)*x[3]))[:5]

---

In [18]:
with open('C:/Users/asus/Downloads/nlp/Lab4/lab4.test.1.txt' ,encoding='UTF-8-sig') as f:
    line = f.readlines()
with open('C:/Users/asus/Downloads/nlp/Lab4/lab4.test.1.txt' ,encoding='UTF-8-sig') as f:
    line2 = f.readlines()
    
with open('C:/Users/asus/Downloads/nlp/Lab4/lab4.test.2.txt' ,encoding='UTF-8-sig') as f:
    line3 = f.readlines()
    
with open('C:/Users/asus/Downloads/nlp/Lab4/lab4.confusables.txt' ,encoding='UTF-8-sig') as f:
    confusable = f.readlines()

In [13]:
"""deal with cinfusable.txt"""
for i in range(len(confusable)):
    confusable[i] =confusable[i].split('\t')
    confusable[i][1] =confusable[i][1].split('\n')

In [14]:
"""deal with test1.txt""" 
# split into single word
for i in range(len(line)):
    line[i] = line[i].split('\t')
    line[i][1] = line[i][1].replace('\n', '')
for i in range(len(line)):
    for j in range(2):
        line[i][j] = line[i][j].split(' ')
""" deal with test1.txt """
# split into sentence
for i in range(len(line2)):
    line2[i] = line2[i].split('\t')
    line2[i][1] = line2[i][1].replace('\n', '')

In [15]:
""" match with confusable list"""
# hit = 0
# for n in range(len(line)):
#     for j in range(len(line[n][0])):
#         for i in range(len(confusable)):
#             #比對confusable, 若存在就用confusable[i][1][0]的取代
#             if line[n][0][j] == confusable[i][0]: 
#                 temp = line[n][0][:-1]
#                 line[n][0][j] = line[n][0][j].replace(line[n][0][j], confusable[i][1][0])
# #                 print("correct: ", line[n][0][j], "wrong: ",confusable[i][0] )
#                 if line[n][0][:-1] == line[n][1]:
#                     hit = hit +1
#                     print("Error: ", line[n][0][j])
#                     print("Candidate: ", confusable[i][0])
#                     print("Correction: ", confusable[i][0])
#                     print(temp, " --> " ,line[n][0])
#                     print("Hits = ", hit)
#                     print()
# #                     print(confusable[i][0],line[n][0][j], line[n][0])

' match with confusable list'

In [16]:
"""NetSpeak API"""
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import requests

API_URL = "http://api.netspeak.org/netspeak3/search?query=%s"
class NetSpeak:
    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0 (compatible; MSIE 5.5; Windows NT)'}
        self.page = None

    def __getPageContent(self, url):
        return requests.get(url, headers=self.headers).text
        # return self.opener.open(url).read()

    def __rolling(self, url, maxfreq=None):
        if maxfreq:
            webdata = self.__getPageContent(url + "&maxfreq=%s" % maxfreq)
        else:
            webdata = self.__getPageContent(url)
        if webdata:
            # webdata = webdata.decode('utf-8')
            results = [data.split('\t') for data in webdata.splitlines()]
            results = [(data[2], float(data[1])) for data in results]
            lastFreq = int(results[-1][1])
            if lastFreq != maxfreq:
                return results + self.__rolling(url, lastFreq)
            else:
                return []
        else:
            return []

    def search(self, query):
        queries = query.split()
        new_query = []
        for token in queries:
            if token.count('|') > 0: 
                new_query.append('[+{0}+]'.format('+'.join(token.split('|'))))
            elif token == '*':
                new_query.append('?')
            else:
                new_query.append(token)
        new_query = '+'.join(new_query)
        url = API_URL % (new_query.replace(' ', '+'))
        return self.__rolling(url)

if __name__ == "__main__":
    SE = NetSpeak()
    # Search_Result = SE.search("approach that *")
    # print Search_Result
    # at * time
    ## for loop
    test = 'when the brake is finished'.split()
    print(test)
    # test = '? is finished'
    # test = 'brake is finished'

    for i in range(len(test) - 2):
        print(i)
        res = SE.search(' '.join(test[i:i + 3]))
        if res:
            print('\n'.join('\t'.join([str(y) for y in x]) for x in res))
        else:
            print('not found')


['when', 'the', 'brake', 'is', 'finished']
0
when the brake	7307.0
1
the brake is	10569.0
2
not found


In [17]:
"""match with big.txt"""
big_txt = ((open("C:/Users/asus/Downloads/nlp/big.txt", encoding='utf-8').read()))
big_txt = big_txt.split()
hit=0

for i in range(len(line)):
    temp=[]
    count=0
    for j in range(len(line[i][0])-1):
# """if not in bit.txt-> replace the word by 'correction' and get the trigram combination"""
        if line[i][0][j] not in big_txt: #Mr J 會有問題 -> +continue 但這樣會少兩個因為有兩句開就是錯的
            if j ==0 and line[i][0][j]=='Mr' : continue
            word = line[i][0][j]
            ori_sent= ' '.join(line[i][0])  
            if line[i][0][j+1] == '':  #字在最後一個
                a= line[i][0][j-2]+ " "+line[i][0][j-1]+" "+correction(word)[0][0]
                b= line[i][0][j-2]+ " "+line[i][0][j-1]+" "+correction(word)[1][0]
                c= line[i][0][j-2]+ " "+line[i][0][j-1]+" "+correction(word)[2][0]
                d= line[i][0][j-2]+ " "+line[i][0][j-1]+" "+correction(word)[3][0]
                e= line[i][0][j-2]+ " "+line[i][0][j-1]+" "+correction(word)[4][0]
                temp.append([a,b,c,d,e])  
#                 ori_sent= ' '.join(line[i][0])
# """Get five kind of trigram and find the max count"""
                candiate = [correction(word)[0][0],correction(word)[1][0],correction(word)[2][0],correction(word)[3][0]
                            ,correction(word)[4][0]]
# """  Get the Max count = word[0]: sentence, word[1]: count """      
                correct_=[]
                for m in range(5):
                    if len(SE.search(((temp[0][m]).lower()))) !=0:
                        correct_.append((SE.search(((temp[0][m]).lower()))))
                        max_count=0
                        for k in range(len(correct_)):
                            if correct_[k][0][1]>max_count:  #find MAX
                                max_count = correct_[k][0][1]
                                max_word = correct_[k][0][0]
#                                 print("max_word", max_word)  #test
                                c_word = max_word.split()[-1] #????
                                line[i][0] = line2[i][0].replace(word,c_word)
                                edit_sent = (line[i][0])
                    elif len(correct_)==0:  #所有組合都不在netspeak中
                        """#放回原本的trigram, 次數為0"""
                        correct_.append([(temp[0][0], int(10.0))])  
                        max_word = correct_[0][0][0]
                        c_word = max_word.split()[-1] #???
                        line[i][0] = line2[i][0].replace(word,c_word)
                        edit_sent = (line[i][0])         
                if edit_sent[:-1] == line2[i][1] :  #最後有空白
                    hit = hit +1     
#                 print("count correction: ", i)
                print("Error: ", word)
                print("Candidate: ", candiate)
                print("Correction: ",c_word) #not yet!
                print( ori_sent , " --> ",edit_sent[:-1])
                print("Hits = ", hit)
                print()
            elif j-1<0: #字在第一個
                a= correction(word)[0][0]+" "+ line[i][0][j+1] +" "+ line[i][0][j+2] 
                b= correction(word)[1][0]+" "+ line[i][0][j+1] +" "+ line[i][0][j+2]
                c= correction(word)[2][0]+ " "+line[i][0][j+1] +" "+ line[i][0][j+2]
                d= correction(word)[3][0]+ " "+line[i][0][j+1] +" "+ line[i][0][j+2]
                e= correction(word)[4][0]+ " "+line[i][0][j+1] +" "+ line[i][0][j+2]
                temp.append([a,b,c,d,e])  
                ori_sent= ori_sent= line[i][0]
                candiate = [correction(word)[0][0],correction(word)[1][0],correction(word)[2][0],correction(word)[3][0]
                            ,correction(word)[4][0]]
                correct_=[]
                for m in range(5):
                    if len(SE.search(((temp[0][m]).lower()))) !=0:
                        correct_.append((SE.search(((temp[0][m]).lower()))))
                        max_count=0
                        for k in range(len(correct_)):
                            if correct_[k][0][1]>max_count:  #find MAX
                                max_count = correct_[k][0][1]
                                max_word = correct_[k][0][0]
                                c_word = max_word.split()[0] #????
                                line[i][0] = line2[i][0].replace(word,c_word)
                                edit_sent = (line[i][0])
                    elif len(correct_)==0:  #所有組合都不在netspeak中
                        correct_.append([(temp[0][0], int(10.0))])  #放回原本的, 次數為0
                        max_word = correct_[0][0][0]
                        c_word = max_word.split()[0] #????
                        line[i][0] = line2[i][0].replace(word,c_word)
                        edit_sent = (line[i][0])
                if edit_sent[:-1] == line2[i][1] :  #最後有空白
                    hit = hit +1      
#                 print("count correction: ", i)
                print("Error: ", word)
                print("Candidate: ", candiate)
                print("Correction: ",c_word) 
                print( ' '.join(ori_sent) , " --> ",edit_sent[:-1])
                print("Hits = ", hit)
                print()   
            else:  #字在中間
                a= line[i][0][j-1]+" "+correction(word)[0][0]+" "+ line[i][0][j+1] #取5個
                b= line[i][0][j-1]+" "+correction(word)[1][0]+" "+ line[i][0][j+1] 
                c= line[i][0][j-1]+" "+correction(word)[2][0]+ " "+line[i][0][j+1] 
                d= line[i][0][j-1]+" "+correction(word)[3][0]+ " "+line[i][0][j+1] 
                e= line[i][0][j-1]+" "+correction(word)[4][0]+ " "+line[i][0][j+1] 
                temp.append([a,b,c,d,e])
                ori_sent= ori_sent= line[i][0]
                candiate = [correction(word)[0][0],correction(word)[1][0],correction(word)[2][0],correction(word)[3][0]
                            ,correction(word)[4][0]]
                correct_=[]
                for m in range(5):
                    if len(SE.search(((temp[0][m]).lower()))) !=0:
                        correct_.append((SE.search(((temp[0][m]).lower()))))
                        max_count=0
                        for k in range(len(correct_)):
                            if correct_[k][0][1]>max_count:  #find MAX
                                max_count = correct_[k][0][1]
                                max_word = correct_[k][0][0]
                                c_word = max_word.split()[1] 
                                line[i][0] = line2[i][0].replace(word,c_word)
                                edit_sent = line[i][0]
                    elif len(correct_)==0:  #所有組合都不在netspeak中
                        correct_.append([(temp[0][0], int(10.0))])  #放回原本的, 次數為0
                        max_word = correct_[0][0][0]
                        c_word = max_word.split()[1] #????
                        line[i][0] = line2[i][0].replace(word,c_word)
                        edit_sent = (line[i][0])
                
                if edit_sent[:-1] == line2[i][1] :  #最後有空白
                    hit = hit +1  
#                 print("count correction: ", i)
                print("Error: ", word)
                print("Candidate: ", candiate)
                print("Correction: ",c_word) #not yet!
                print(' '.join(ori_sent)[:-1] , " --> ",edit_sent[:-1])
                print("Hits = ", hit)
                print()   
                break  

Error:  strang
Candidate:  ['strange', 'strong', 'staring', 'sprang', 'string']
Correction:  strange
I felt very strang   -->  I felt very strange
Hits =  1

Error:  brack
Candidate:  ['black', 'back', 'branch', 'brick', 'break']
Correction:  back
when the brack was finished  -->  when the back was finished
Hits =  1

Error:  weanter
Candidate:  ['winter', 'water', 'weather', 'wanted', 'wander']
Correction:  water
in the weanter when it was snowing  -->  in the water when it was snowing
Hits =  1

Error:  gost
Candidate:  ['just', 'most', 'ghost', 'got', 'goes']
Correction:  most
I thought it was a gost   -->  I thought it was a most
Hits =  1

Error:  steped
Candidate:  ['stepped', 'stooped', 'striped', 'stupid', 'stopped']
Correction:  stepped
when I first steped   -->  when I first stepped
Hits =  2

Error:  exclation
Candidate:  ['exclusion', 'excavation', 'exaltation', 'exultation', 'exudation']
Correction:  exclusion
I was on an exclation   -->  I was on an exclusion
Hits =  2

E

Error:  barth
Candidate:  ['birth', 'both', 'part', 'bath', 'baths']
Correction:  part
so I had a barth   -->  so I had a part
Hits =  29

Error:  clouths
Candidate:  ['clothes', 'cloths', 'sloughs', 'clouds', 'clots']
Correction:  clothes
I made clouths with it  -->  I made clothes with it
Hits =  30

Error:  skeard
Candidate:  ['steward', 'seward', 'stead', 'seared', 'heard']
Correction:  heard
I was skeard of it  -->  I was heard of it
Hits =  30

Error:  exsiting
Candidate:  ['exciting', 'existing', 'excising', 'editing', 'expiring']
Correction:  exciting
one of my exsiting days  -->  one of my exciting days
Hits =  31

Error:  crames
Candidate:  ['crimes', 'crime', 'came', 'comes', 'cramped']
Correction:  crime
found some crames and ate them  -->  found some crime and ate them
Hits =  31

Error:  carful
Candidate:  ['careful', 'harmful', 'lawful', 'caseful', 'carpal']
Correction:  careful
I was very carful   -->  I was very careful
Hits =  31

Error:  gost
Candidate:  ['just', 'mo

Hits =  55

Error:  emtied
Candidate:  ['emptied', 'enticed', 'untied', 'emitted', 'eddied']
Correction:  emptied
have to be emtied every day  -->  have to be emptied every day
Hits =  56

Error:  belive
Candidate:  ['believe', 'believed', 'belief', 'believes', 'beloved']
Correction:  believes
he belive in the old story  -->  he believes in the old story
Hits =  57

Error:  fameous
Candidate:  ['famous', 'fameuse', 'caseous', 'gaseous', 'famos']
Correction:  famous
the most fameous player  -->  the most famous player
Hits =  58

Error:  centery
Candidate:  ['century', 'center', 'gentry', 'centers', 'centre']
Correction:  center
of the centery   -->  of the center
Hits =  58

Error:  smuggle
Candidate:  ['smuggled', 'struggle', 'smudge', 'smuggl', 'zmuggl']
Correction:  struggle
the man had tired to smuggle   -->  the man had tired to struggle
Hits =  58

Error:  petroil
Candidate:  ['petropol', 'patrol', 'petrol', 'peril', 'petrous']
Correction:  petrol
put more petroil   -->  put more

Error:  occured
Candidate:  ['occurred', 'occur', 'obscured', 'occupied', 'occurs']
Correction:  occurs
when it occured   -->  when it occurs
Hits =  90

Error:  wether
Candidate:  ['whether', 'weather', 'water', 'whither', 'father']
Correction:  whether
she asked us wether   -->  she asked us whether
Hits =  91

Error:  extremly
Candidate:  ['extremely', 'extremity', 'extreme', 'extremes', 'extrmly']
Correction:  extremely
an extremly hot day  -->  an extremely hot day
Hits =  92

Error:  oppinion
Candidate:  ['opinion', 'opinions', 'oppinion', 'oppnion', 'oppinon']
Correction:  opinions
their oppinion about  -->  their opinions about
Hits =  92

Error:  wat
Candidate:  ['what', 'with', 'was', 'want', 'wait']
Correction:  what
to think wat   -->  to think what
Hits =  93

Error:  contenue
Candidate:  ['continued', 'continue', 'continues', 'contented', 'content']
Correction:  continue
to contenue with  -->  to continue with
Hits =  94

Error:  thout
Candidate:  ['though', 'that', 'thro

---

In [ ]:
# find the MAX
for i in range(2):
    temp=0
    if correct_[i][0][1]>temp:
        temp = correct_[i][0][1]
        word = correct_[i][0]
print(word)

In [ ]:
"""split sentence into tri-gram"""
def trigram(text):
    trigram_word=[]
    trigram = (ngrams(text.split(), 3))
    for text in trigram:
        trigram_word.append(" ".join(text))
    return trigram_word